# Подготавливаем данные

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv('/Users/mamolchanova/Documents/Обучение/ML/18 занятие - рекомендации на основе скрытых факторов/hetrec2011-delicious-2k/user_taggedbookmarks-timestamps.dat', sep='\t', engine='python')

In [4]:
df.head(20)

,userID,bookmarkID,tagID,timestamp
0,8,1,1,1289255362000
1,8,2,1,1289255159000
2,8,7,1,1289238901000
3,8,7,6,1289238901000
4,8,7,7,1289238901000
5,8,8,1,1289238545000
6,8,8,8,1289238545000
7,8,8,9,1289238545000
8,8,9,1,1289237833000
9,8,9,10,1289237833000


In [5]:
df_bookmarks = pd.read_csv('/Users/mamolchanova/Documents/Обучение/ML/18 занятие - рекомендации на основе скрытых факторов/hetrec2011-delicious-2k/bookmarks.dat', sep='\t', engine='python')

In [6]:
df_bookmarks.head()

,id,md5,title,url,md5Principal,urlPrincipal
0,1,ab4954b633ddaf5b5bba6e9b71aa6b70,IFLA - The official website of the Internation...,http://www.ifla.org/,7f431306c428457bc4e12b15634484f,www.ifla.org
1,2,2221e9cd106d269dd34682666f576fa3,gcdp-e.pdf (application/pdf Object),http://archive.ifla.org/VII/s14/nd1/gcdp-e.pdf,1ef8cfcfe968101fa9b4e301847503d4,archive.ifla.org
2,7,c97c571dadaddbbb493126a0d4d01ba3,EdSelect,http://www.edselect.com/,792fd7eb20143386d0c4eb193c6124d,www.edselect.com
3,8,25bfe8dca0ef263ec9c341b9f16c38b5,Cool Canada (Collections Canada),http://www.collectionscanada.gc.ca/cool/index-...,6fce4f6391516f0732531d9cfacda5b7,www.collectionscanada.gc.ca
4,9,c97284629e17b8e2861afaacd59918bc,Kidsreads.com,http://www.kidsreads.com/,5854ce8404857a45373eea01a3d98000,www.kidsreads.com


In [7]:
del df['tagID']
del df['timestamp']

In [8]:
df['count']=1

In [9]:
df_group = df.groupby(['userID', 'bookmarkID']).agg({"count": "sum"})

In [10]:
df_group.head()

count
userID bookmarkID       
8      1               1
       2               1
       7               3
       8               3
       9               2

In [11]:
import scipy
import numpy as np

In [12]:
df_group = df_group.reset_index()
df_group.head(10)

,userID,bookmarkID,count
0,8,1,1
1,8,2,1
2,8,7,3
3,8,8,3
4,8,9,2
5,8,10,2
6,8,11,3
7,8,14,5
8,8,15,2
9,8,16,2


In [42]:
#Меняю у данных индексы, чтобы правильно сформировать их в матрицу

In [13]:
INDEX_TO_USER = df_group['userID'].unique().tolist()
USER_TO_INDEX = {v: i for i,v in enumerate(INDEX_TO_USER)}

In [14]:
INDEX_TO_BOOKMARK = df_group['bookmarkID'].unique().tolist()
BOOKMARK_TO_INDEX = {v: i for i,v in enumerate(INDEX_TO_BOOKMARK)}

In [15]:
df_group['user_id'] = df_group.apply(lambda r: USER_TO_INDEX[r['userID']], axis=1)

In [16]:
df_group['bookmark_id'] = df_group.apply(lambda r: BOOKMARK_TO_INDEX[r['bookmarkID']], axis=1)

In [17]:
df_group.head()

,userID,bookmarkID,count,user_id,bookmark_id
0,8,1,1,0,0
1,8,2,1,0,1
2,8,7,3,0,2
3,8,8,3,0,3
4,8,9,2,0,4


In [18]:
values = df_group['count'].tolist()
rows = df_group['user_id'].tolist()
columns = df_group['bookmark_id'].tolist()

In [19]:
import scipy.sparse as sp 

In [28]:
user_data = sp.csr_matrix((values, (rows, columns)), shape=(len(INDEX_TO_USER), len(INDEX_TO_BOOKMARK))).T

# Готовим модель

In [29]:
import implicit

In [30]:
model = implicit.als.AlternatingLeastSquares(factors=50)

In [31]:
model.fit(user_data)

In [32]:
userid=0

user_items = user_data.T.tocsr()
recommendations = model.recommend(userid, user_items)

In [41]:
for bookmark, score in recommendations:
    print (INDEX_TO_BOOKMARK[bookmark], score)

29077 0.22512837
4638 0.22488521
29093 0.2181431
14121 0.2023558
29103 0.20210467
29105 0.19928211
1631 0.19811617
29110 0.1979307
29104 0.19301772
2178 0.18969792
